In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import requests
import PIL
from PIL import Image
import os
from torchvision.transforms import Normalize, Compose, Resize, ToTensor

In [ ]:
!git clone https://github.com/xinyu1205/recognize-anything.git

%cd recognize-anything
!git checkout ec6b4241c5036e337a4543838deb9bff4990de97

Cloning into 'recognize-anything'...
remote: Enumerating objects: 663, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 663 (delta 260), reused 241 (delta 195), pack-reused 287
Receiving objects: 100% (663/663), 27.11 MiB | 30.11 MiB/s, done.
Resolving deltas: 100% (345/345), done.
/content/recognize-anything
Note: switching to 'ec6b4241c5036e337a4543838deb9bff4990de97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ec6b424 update


In [ ]:
!pip install timm transformers fairscale pycocoevalcap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 35.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.7 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332104 sha256=812c922f997a7d0cdb208ec81e579680c0c98f2d6e26c169f3b8ccc90aa0d5a1
  Stored in directory: /root/.cache/pip/wheels/78/a4/c0/fb0a7ef03cff161611c3fa40c6cf898f76e58ec421b88e8cb3
Successfully built fairscale


In [ ]:
from ram.models import ram
from ram import inference_ram as inference

In [ ]:
def convert_to_rgb(image):
    return image.convert("RGB")
def get_transform(image_size=384):
    return Compose([
        convert_to_rgb,
        Resize((image_size, image_size)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
if not os.path.exists('pretrained'):
    os.makedirs('pretrained')
ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
if not os.path.exists(ram_weights_path):
    !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
else:
    print("RAM weights already downloaded!")

--2023-12-09 17:53:21--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 54.230.18.84, 54.230.18.85, 54.230.18.110, ...
Connecting to huggingface.co (huggingface.co)|54.230.18.84|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2023-12-09 17:53:21--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_14m.pth%22%3B&Expi

In [ ]:
transform = get_transform()

In [ ]:
model = ram(pretrained='pretrained/ram_swin_large_14m.pth',
            vit='swin_l',
            image_size=384)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l


In [ ]:
model.eval()

RAM(
  (visual_encoder): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        dim=192, input_resolution=(96, 96), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=192, input_resolution=(96, 96), num_heads=6, window_size=12, shift_size=0, mlp_ratio=4.0
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=192, window_size=(12, 12), num_heads=6
              (qkv): Linear(in_features=192, out_features=576, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=192, out_features=192, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-

In [ ]:
model = model.to(device)

In [ ]:
images_dir = "images/demo"
image_files = [f"{images_dir}/{file}" for file in sorted(os.listdir(images_dir)) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [ ]:
image_files[0]

'images/demo/demo1.jpg'

In [ ]:
image = transform(Image.open(image_files[0])).unsqueeze(0).to(device)

In [ ]:
res = inference(image, model)

In [ ]:
print("Image Tags: ", res[0])

Image Tags:  armchair | blanket | lamp | carpet | couch | dog | floor | furniture | gray | green | living room | picture frame | pillow | plant | room | sit | stool | wood floor


In [ ]:
url = "https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=large"
img = transform(Image.open(requests.get(url, stream=True).raw)).unsqueeze(0).to(device)

In [ ]:
res = inference(img, model)

In [ ]:
print(res[0].replace(" | ", ", "))

beverage, burrito, coffee, drink, food, milkshake, sandwich


In [ ]:
def img_to_tags(img_url):
    try:
        imag = transform(Image.open(requests.get(img_url, stream=True).raw)).unsqueeze(0).to(device)
        results = inference(imag, model)
        return results[0].replace(" | ", ", ")
    except PIL.UnidentifiedImageError:
        return np.nan

In [ ]:
print(img_to_tags(url))

beverage, burrito, coffee, drink, food, milkshake, sandwich


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving image_df.csv to image_df.csv


In [ ]:
df = pd.read_csv('image_df.csv')

id                                                    46
url    https://pbs.twimg.com/tweet_video_thumb/EPFYz1...
Name: 45, dtype: object

In [ ]:
temp_df = df[0:10]
temp_df

,id,url
0,1,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...
1,2,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...
2,3,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...
3,4,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...
4,5,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...
5,6,https://pbs.twimg.com/amplify_video_thumb/1322...
6,7,https://pbs.twimg.com/media/EHo1yZyU0AIBW8N?fo...
7,8,https://pbs.twimg.com/amplify_video_thumb/1019...
8,9,https://pbs.twimg.com/media/D2qiPBTX0AI_13F?fo...
9,10,https://pbs.twimg.com/media/EeT6pQZVoAAM2Lj?fo...


In [ ]:
temp_df['image_tags'] = temp_df['url'].apply(img_to_tags)

In [ ]:
df['image_tags'] = df['url'].apply(img_to_tags)

In [ ]:
df.to_csv("./final_df.csv", index=False)

'https://pbs.twimg.com/amplify_video_thumb/1322382278794858496/img/eKRa2rXniZ_eOCNB.jpg'

In [ ]:
files.download("final_df.csv")